In [1]:
from aiatools import *
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import os
import seaborn as sns
import matplotlib.pyplot as plt
import re
from zipfile import ZipFile, BadZipFile

In [ ]:
# reading the csv provided by Technovation
df = pd.read_csv("Technovation.csv")
df['Source code url']

In [ ]:
# This is to drop all of the rows that don't have app inventor projects in them.
first = df[df['Source code url'].isna() == False]

# resetting the index so we can see how many app inventor uploads there are.
first = first[['Source code url']].reset_index(drop=True)

# Just renaming the columns
first = first.rename(columns={'Source code url' : 'Source'})

# The first project was not valid, as it was a test entry by the Technovation team, so I excluded that.
first = first.iloc[1:, :]

first['Source'].get(3)

In [ ]:
# In this cell, we want to isolate the rows that have app inventor projects that have the file type ".aia" because the other zip files are not analyzable with the aiatools package.
pattern = r"(\.aia)$"
second = first[first['Source'].str.contains(pattern, regex=True) == True].reset_index(drop=True)
second

In [ ]:
# I want to automate the downloading for the files, so I don't have to download 880 files manually.

# DANGEROUS CODE BELOW: TO DOWNLOAD THE FILES, UNCOMMENT THE CODE BELOW AND RUN
# IMPORTANT: DO NOT RUN THIS CELL MULTIPLE TIMES.

# options = webdriver.ChromeOptions()
# download_directory = './aia_downloads'
# options.add_experimental_option("prefs", {
#     "download.default_directory": download_directory,
#     "download.prompt_for_download": False,
#     "download.directory_upgrade": True,
#     "safebrowsing.enabled": True
# })
# driver = webdriver.Chrome(options=options)

# urls = second['Source'].tolist()
# driver.get(urls[0])
# for i, url in enumerate(urls):
#     driver.get(url)

In [ ]:
# Plan of Action:

# 1: Determine the total number of screens, do a graphical analysis of that; find the total per project and total overall. Max? Min? <5, 10+, 20+  

# 2: Then we can examine the total amount of blocks and blocks per screen, and do an analysis on that too.

# 3: Next we want to sort by the COMPONENTS by its complexity and do a general analysis of that. How many projects used which 'level' of Components?
#    The previous analysis separated the blocks using 2 measures: Sensor Components and Functionality.

# 4: We can then do a close analysis on the complexity of the components, possibly assign a 'Component Complexity Score' to each project.

# 5: Then we can do the same thing but with BLOCKS this time.

# 6: We can also then analyze extensions. How many projects used extensions? Of these projects, are there any 'storybook' projects? What is the comparison of overall score of the projects 
#    that used extensions and those that didn't?

# 7: Lastly, we can analyse the use of AI. Did projects use AI? How many did? What are the scores for that? How can we categorize the usage of AI with the other blocks? What is the comparison of 
#    overall project score compared to the ones that didn't use AI?

# 8: *Potential Analysis* - Develop a rubric and assign each assignment a 'Final Complexity Score', then use that to analyze the dynamic between complexity score and placement/actual scores
#    in the Technovation competition.

In [ ]:
# Execution of Step 1: Analysis on Screens

folder_path = "./aia_downloads"
projects_list = []

for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):

        try:
            projects_list.append(AIAFile(os.path.join(folder_path, filename)))
        except (KeyError, OSError, BadZipFile):
            pass

# We got a total of 870 successful reads on the projects
len(projects_list)

In [ ]:
# Data Cleaning

In [ ]:
# after some investigation, these two projects both had no components in their entire project

# Finding out total number of screens out of 870 projects
total_screens = []
all_screens = []
project_screen_dict = {}
for project in projects_list:
    all_screens += project.screens
    total_screens.append(len(project.screens))
    project_screen_dict[re.search(r"([^\/]+)\.aia", project.filename).group(1)] = len(project.screens)
data = pd.DataFrame({'Project Name': project_screen_dict.keys(), 'Screen Count': project_screen_dict.values()})

print(sum(total_screens))

avg_screen_count = np.sum(total_screens) / len(total_screens)

median_screen_count = np.median(total_screens)

max_screen_count = max(data['Screen Count'])

min_screen_count = min(data[data['Screen Count'] != 0]['Screen Count'])

screen_distribution = data.groupby('Screen Count')[['Project Name']].agg('count').rename(columns={'Project Name' : 'Number of Projects'}).reset_index()

sns.histplot(data['Screen Count'])
plt.ylabel('# of Projects')
plt.xlabel('# of Screens per Project')
plt.title('Distribution of Screens per Project')

In [ ]:
# Now we can check how many screens had no components are them / how many screens were unused.

project_screens_components = []

for project in projects_list:
    screens = project.screens
    for screen in screens:
        project_screens_components.append(screen.components)

# project_screens_components[0]

# From the code above, we can see that the first element in a screen list is just the screen itself. 
# Therefore, we will say that if the screen list has a length of 1, that means there are no components in that screen

mt_screens_dict = {}
num_projects_with_empty_screens = 0
for project in projects_list:
    screens = project.screens
    # Checks how many empty screens there are in the project.
    mt = 0
    for screen in screens:
        if len(screen.components) == 1:
            mt += 1
    if mt > 0:
        num_projects_with_empty_screens += 1
    mt_screens_dict[re.search(r"([^\/]+)\.aia", project.filename).group(1)] = mt

num_projects_with_empty_screens = len(mt_screens_dict)


# Create a dataframe to visualize the project to empty screens ratios.
p = {'project name': mt_screens_dict.keys(), 'Empty Screen Count': mt_screens_dict.values(), 'Project File': projects_list}
empty = pd.DataFrame(p)

total_empty_screens =  np.sum(empty['Empty Screen Count'])
empty

In [ ]:
# Invalid Submissions (no screens)
invalid = []
for project in projects_list:
    if len(project.screens) == 0:
        invalid.append(project.filename)
        
# next we will see if there are any submissions that have no components.
for project in projects_list:
    no_components = True
    for screen in project.screens:
        # we set this = 1 because the screen itself is always included into the components list.
        if len(screen.components) > 1:
            no_components = False
    if no_components:
        if not project.filename in invalid:
            invalid.append(project.filename)
invalid
# There are 2.

In [ ]:
# The idea of Storybook project can be categorized as 20+ screens. We can check for the projects that have 20+ screens

story_books = data[data["Screen Count"] >= 20]
num_story_books = story_books.shape[0]
num_story_books

In [ ]:
# Now lets combine all these into one dataframe
Combined_data = data.merge(right=empty, left_on='Project Name', right_on='project name')
Combined_data = Combined_data[['Project Name', 'Project File', 'Screen Count', 'Empty Screen Count']]
Combined_data['Storybook 20-29'] = ((data["Screen Count"] >= 20) & (data["Screen Count"] < 30))
Combined_data['Storybook 30-39'] = ((data["Screen Count"] >= 30) & (data["Screen Count"] < 40))
Combined_data['Storybook 40-50'] = ((data["Screen Count"] >= 40) & (data["Screen Count"] <= 50))
screen_data = (avg_screen_count, max_screen_count, min_screen_count)
Combined_data
# First step done

In [ ]:
# Story Book Projects Visualizations

num_20_to_29_storybooks = len(Combined_data[Combined_data['Storybook 20-29'] == True])
num_30_to_39_storybooks = len(Combined_data[Combined_data['Storybook 30-39'] == True])
num_40_plus_storybooks = len(Combined_data[Combined_data['Storybook 40-50'] == True])
projects_20_to_29 = Combined_data[Combined_data['Storybook 20-29'] == True]
projects_30_to_39 = Combined_data[Combined_data['Storybook 30-39'] == True]
projects_40_to_50 = Combined_data[Combined_data['Storybook 40-50'] == True]

plt.figure(figsize=(8, 7))
sns.barplot(data=projects_20_to_29.sort_values('Screen Count', ascending=True), x='Screen Count', y='Project Name', width = 1, hue='Project Name', palette='light:#10ABCD')
plt.title('Possible Storybook Projects w/ 20-29 Screens')

plt.figure(figsize=(8, 7))
sns.barplot(data=projects_30_to_39.sort_values('Screen Count', ascending=True), x='Screen Count', y='Project Name', width = 1, hue='Project Name', palette='light:#10ABCD')
plt.title('Possible Storybook Projects w/ 30-39 Screens')

plt.figure(figsize=(8, 7))
sns.barplot(data=projects_40_to_50.sort_values('Screen Count', ascending=True), x='Screen Count', y='Project Name', width = 1, hue='Project Name', palette='light:#10ABCD')
plt.title('Possible Storybook Projects w/ 40-50 Screens')

plt.figure(figsize=(8, 8))
sns.barplot(data=story_books.sort_values('Screen Count', ascending=True), x='Screen Count', y='Project Name', width = 1, hue='Project Name', palette='light:#10ABCD')
plt.title('All Possible Storybook Projects')

In [ ]:
# Analysis on components

total_components_list = []
for project in projects_list:
    num = 0
    for screen in project.screens:
        # We subtract one here because the first component is just the screen itself.
        num += len(screen.components) - 1
    total_components_list.append(num)

# We want to record this onto out dataset
Combined_data['Components Count'] = total_components_list


project_components = {}
for project in projects_list:
    screens_dict = {}
    for screen in project.screens:
        screens_dict[screen] = screen.components
    project_components[re.search(r"([^\\]+)\.aia", project.filename).group(1)] = screens_dict
    
Average_num_components_per_project = np.mean(total_components_list)
print(sum(total_components_list))
# Note: This average includes the screens with no components in there.
Average_num_components_per_screen = sum(total_components_list) / sum(total_screens)
Average_num_components_per_screen

In [ ]:
# For this average, we need to exclude the screens without any components, or else it might skew the distribution

non_empty_screens_list = []
for project in projects_list:
    for screen in project.screens:
        if len(screen.components) > 1:
            non_empty_screens_list.append(screen.components)
# we have to subtract 1 here because the screen is included as a component itself.
non_empty_components = [len(screen) - 1 for screen in non_empty_screens_list]

Average_num_components_per_screen = sum(non_empty_components) / len(non_empty_screens_list)
Average_num_components_per_screen

In [ ]:
# Complexity categorization for components

lv_1_components = ['Image', 'Label', 'TextBox', 'Button']

lv_2_components = ['Camcorder','Camera','VideoPlayer','Translator','SoundRecorder',
        'Player','Sound','ContactPicker','EmailPicker','PhoneNumberPicker',
        'PhoneCall','Texting','WebViewer','Pedometer','Clock','LocationSensor',
        'BarcodeScanner','Variable','TinyDB','Checkbox','ListPicker','ListView',
        'Notifier','PasswordTextBox','Slider','Spinner','Switch','HorizontalArrangement',
        'VerticalArrangement','TableArrangement','HorizontalScrollArrangement','VerticalScrollArrangement']

lv_3_components = ['ImagePicker','SpeechRecognizer','TextToSpeech','Sharing',
        'Twitter','ActivityStarter','Maps','AccelerometerSensor',
        'GyroscopeSensor','ProximitySensor','NearField','OrientationSensor','TimePicker']

lv_4_components = ['Web','Bluetooth Client','BluetoothServer',
        'Chart', 'ChartData2D','Serial','MagneticFieldSensor',
        'Barometer','Hygrometer','LightSensor','Thermometer',
        'Spreadsheet','CloudDB','FirebaseDB','BluetoothLE']

lv_5_components = ['PersonalAudioClassifier', 'PersonalImageClassifier', 'LookExtension']


def level_sorting_components_func(level, project):
    """
    Returns a list of the specific components of a given project in the given level of complexity.

    parameters:
        level - list of components in a level.
        project - AIAFile of a project.
    """
    
    list = []
    for screen in project.screens:
        for component in screen.components:
            if component.type() in level:
                list.append(component)
    return list

    
level_1_sorting_components = [len(level_sorting_components_func(lv_1_components, project)) for project in projects_list]

level_2_sorting_components = [len(level_sorting_components_func(lv_2_components, project)) for project in projects_list]

level_3_sorting_components = [len(level_sorting_components_func(lv_3_components, project)) for project in projects_list]

level_4_sorting_components = [len(level_sorting_components_func(lv_4_components, project)) for project in projects_list]

level_5_sorting_components = [len(level_sorting_components_func(lv_5_components, project)) for project in projects_list]

Combined_data['Level 1 Components'] = level_1_sorting_components
Combined_data['Level 2 Components'] = level_2_sorting_components
Combined_data['Level 3 Components'] = level_3_sorting_components
Combined_data['Level 4 Components'] = level_4_sorting_components
Combined_data['AI'] = level_5_sorting_components

# We have to check for Extensions separately

def find_extensions(project):
    """
    Returns a list of the components that are extensions.

    parameters:
        level - list of components in a level.
        project - AIAFile of a project.
    """
    list = []
    for screen in project.screens:
        for extension in screen.components(external == True):
            list.append(extension)
    return list

Combined_data['Extensions'] = [len(find_extensions(project)) for project in projects_list]


def condition(column):
    if column >= 1:
        return 1.5
    else:
        return 1

# Now let's look at the complexity score of the components

Combined_data['Level 1 Component Score'] = Combined_data['Level 1 Components'] * 1 * 0.05
Combined_data['Level 2 Component Score'] = Combined_data['Level 2 Components'] * 2 * 0.15
Combined_data['Level 3 Component Score'] = Combined_data['Level 3 Components'] * 3 * 0.3
Combined_data['Level 4 Component Score'] = Combined_data['Level 4 Components'] * 4 * 0.5
Combined_data['AI Score'] = Combined_data['AI'].apply(condition)
Combined_data['Extensions Score'] = Combined_data['Extensions'].apply(condition)

# Changed Rubric to 'Whether or not the projects used AI or Extensions' instead of 'how many'.
Combined_data['Extensions'] = Combined_data['Extensions'] > 0
Combined_data['AI'] = Combined_data['AI'] > 0


Combined_data['Component Complexity Score'] = np.round(
                                              (((Combined_data['Level 1 Component Score'] +
                                              Combined_data['Level 2 Component Score'] +
                                              Combined_data['Level 3 Component Score'] +
                                              Combined_data['Level 4 Component Score']) *
                                              Combined_data['AI Score']) *
                                              Combined_data['Extensions Score'])
                                              )



max_component_score = max(Combined_data['Component Complexity Score'])

min_component_score = min(Combined_data['Component Complexity Score'])


In [ ]:
# Table Visualization of the # of Components and the Component Complexity Score.

Combined_data[['Project Name', 'Level 1 Components', 'Level 2 Components', 'Level 3 Components', 'Level 4 Components', 'AI', 'Extensions', 'Component Complexity Score']].sort_values('Component Complexity Score', ascending=False).head(10)

In [ ]:
sns.scatterplot(data=Combined_data, x='Component Complexity Score', y='Components Count')
plt.ylabel('# of Components')
plt.xlabel('Component Complexity Score')
plt.title('# of Components vs Component Complexity Score')

In [ ]:
sns.scatterplot(data=Combined_data[Combined_data['Level 1 Components'] > 0], x='Component Complexity Score', y='Level 1 Components')
plt.ylabel('# of Level 1 Components')
plt.xlabel('Component Complexity Score')
plt.title('Level 1 Components vs Component Complexity Score')

In [ ]:
sns.scatterplot(data=Combined_data[Combined_data['Level 2 Components'] > 0], x='Component Complexity Score', y='Level 2 Components')
plt.ylabel('# of Level 2 Components')
plt.xlabel('Component Complexity Score')
plt.title('Level 2 Components vs Component Complexity Score')

In [ ]:
sns.scatterplot(data=Combined_data[Combined_data['Level 3 Components'] > 0], x='Component Complexity Score', y='Level 3 Components')
plt.ylabel('# of Level 3 Components')
plt.xlabel('Component Complexity Score')
plt.title('Level 3 Components vs Component Complexity Score')

In [ ]:
sns.scatterplot(data=Combined_data[Combined_data['Level 4 Components'] > 0], x='Component Complexity Score', y='Level 4 Components')
plt.ylabel('# of Level 4 Components')
plt.xlabel('Component Complexity Score')
plt.title('Level 4 Components vs Component Complexity Score')

In [ ]:
sns.scatterplot(data=Combined_data[Combined_data['AI'] > 0], x='Component Complexity Score', y='AI')
plt.ylabel('# of AI Components')
plt.xlabel('Component Complexity Score')
plt.title('AI vs Component Complexity Score')

In [ ]:
sns.scatterplot(data=Combined_data[Combined_data['Extensions'] > 0], x='Component Complexity Score', y='Extensions')
plt.ylabel('# of Extensions')
plt.xlabel('Component Complexity Score')
plt.title('# of Extensions vs Component Complexity Score')

In [ ]:
# Heatmap of Component Correlations

sns.heatmap(data=Combined_data[['Level 1 Component Score', 'Level 2 Component Score', 'Level 3 Component Score', 'Level 4 Component Score', 'Component Complexity Score']].corr(), annot=True)
plt.title('Correlation between Different Levels of Component Scores and Overall Component Complexity Score')

In [ ]:
# More incomplete/invalid submissions.

Combined_data[Combined_data['Components Count'] == 1]
Combined_data[Combined_data['Components Count'] == 1]

# Use the code below to see that the single components in these projects are just labels or a webviewer that sends the user to a website. (num is from 0 - 4, specifying each project)
# num = 0
# Combined_data[Combined_data['Components Count'] == 1]['Project File'].values[num].screens[0].components

In [ ]:
# Now we can see some statistics.

# How many projects used which level of block?
projects_lv1 = Combined_data[Combined_data['Level 1 Components'] > 0].shape[0]
projects_lv2 = Combined_data[Combined_data['Level 2 Components'] > 0].shape[0]
projects_lv3 = Combined_data[Combined_data['Level 3 Components'] > 0].shape[0]
projects_lv4 = Combined_data[Combined_data['Level 4 Components'] > 0].shape[0]

# How many projects used AI?
projects_AI = Combined_data[Combined_data['AI'] > 0].shape[0]

# How many projects used extensions?
projects_ext = Combined_data[Combined_data['Extensions'] > 0].shape[0]

# How many projects used extensions but were storybook style?
project_ext_storybook_20 = len(Combined_data[(Combined_data['Storybook 20-29'] == True) & (Combined_data['Extensions'] > 0)])
project_ext_storybook_30 = len(Combined_data[(Combined_data['Storybook 30-39'] == True) & (Combined_data['Extensions'] > 0)])
project_ext_storybook_40 = len(Combined_data[(Combined_data['Storybook 40-50'] == True) & (Combined_data['Extensions'] > 0)])

project_ext_storybook = project_ext_storybook_20 + project_ext_storybook_30 + project_ext_storybook_40
project_ext_storybook

In [ ]:
# Analysis on Blocks
from project_blocks import *


# How many total blocks are in this project?
blocks_count = []
all_blocks = []
blocks_by_project = []
for project in projects_list:
    blocks_count.append(project.blocks().count())
    blocks_by_project.append([block.type for block in project.blocks])
    for block in project.blocks:
        all_blocks.append(block.type)

print(sum(blocks_count))

Combined_data['Blocks Count'] = blocks_count
Combined_data['Blocks'] = blocks_by_project

# Here, we have to exclude the project named #CONFIDENTIAL. This is the project with nothing, so I can't divide 0 by 0 screens. 
average_num_blocks_per_screen = []
for project in projects_list:
    if re.search(r"([^\/]+)\.aia", project.filename).group(1) != #CONFIDENTIAL:
        average_num_blocks_per_screen.append(np.round(mean_blocks_per_screen(project.screens)).astype('int'))
    
Combined_data['Average Blocks Per Screen'] = average_num_blocks_per_screen

max_blocks_count = max(blocks_count)
min_blocks_count = min(blocks_count)
median_blocks_count = np.median(blocks_count)
average_num_of_blocks_total = sum(blocks_count) / sum(total_screens)


blocks_series = pd.Series(all_blocks)
unique_blocks = pd.Series(all_blocks).unique()

if_statements = blocks_series.str.contains('if')
unique_ifs = blocks_series.to_frame()[blocks_series.str.contains('if')].loc[:, 0].unique()

dictionaries = blocks_series.str.contains('dictionaries')
unique_dicts = blocks_series.to_frame()[blocks_series.str.contains('dictionaries')].loc[:, 0].unique()

lists = blocks_series.str.contains('list')
unique_lists = blocks_series.to_frame()[blocks_series.str.contains('list')].loc[:, 0].unique()


decs = blocks_series.str.contains('declaration')
unique_decs = blocks_series.to_frame()[blocks_series.str.contains('declaration')].loc[:, 0].unique()

forloops = blocks_series.str.contains('for')
unique_forloops = blocks_series.to_frame()[blocks_series.str.contains('for')].loc[:, 0].unique()

procedures = blocks_series.str.contains('procedure')
unique_procedures = blocks_series.to_frame()[blocks_series.str.contains('procedures')].loc[:, 0].unique()

component_blocks = blocks_series.str.contains('component')
unique_component_blocks = blocks_series.to_frame()[blocks_series.str.contains('component')].loc[:, 0].unique()

lv_1_blocks = ['controls_openAnotherScreen', 'lexical_variable_get', 
               'lexical_variable_set', 'controls_openAnotherScreenWithStartValue', 
               'controls_closeApplication', 'controls_closeScreen', 'controls_getStartValue',
              'controls_getPlainStartText', 'controls_closeScreenWithValue', 
               'controls_closeScreenWithPlainText', 'controls_send_to_block', 'component_set_get']

lv_2_blocks = ['local_declaration_statement', 'global_declaration',
       'local_declaration_expression', 'controls_if', 'component_event', 
               'component_component_block', 'component_method']

lv_3_blocks = ['lists_is_in', 'lists_create_with', 'lists_pick_random_item',
       'lists_add_items', 'lists_remove_item', 'lists_position_in',
       'lists_length', 'lists_select_item', 'lists_from_csv_table',
       'lists_is_empty', 'lists_reverse', 'lists_sort',
       'lists_from_csv_row', 'lists_lookup_in_pairs', 'math_on_list',
       'lists_to_csv_table', 'lists_replace_item', 'lists_is_list',
       'lists_join_with_separator', 'lists_to_csv_row', 'lists_copy',
       'lists_maximum_value', 'lists_map', 'lists_insert_item',
       'lists_append_list', 'controls_forEach', 'controls_forRange',
       'controls_for_each_dict', 'controls_while']

lv_4_blocks = ['dictionaries_create_with', 'dictionaries_lookup',
       'dictionaries_set_pair', 'dictionaries_is_key_in',
       'dictionaries_copy', 'dictionaries_recursive_set',
       'dictionaries_recursive_lookup', 'dictionaries_is_dict',
       'dictionaries_getters', 'dictionaries_delete_pair',
       'dictionaries_length', 'dictionaries_get_values',
       'dictionaries_combine_dicts', 'procedures_callnoreturn', 'procedures_defnoreturn',
       'procedures_defreturn', 'procedures_callreturn', 'controls_do_then_return']

def level_sorting_blocks_func(level, project):
    """
    Returns a list of the specific components of a given project in the given level of complexity.

    parameters:
        level - list of components in a level.
        project - AIAFile of a project.
    """
    
    list = []
    for block in project.blocks():
        if block.type in level:
            list.append(block)
    return list

level_1_sorting_blocks = [len(level_sorting_blocks_func(lv_1_blocks, project)) for project in projects_list]

level_2_sorting_blocks = [len(level_sorting_blocks_func(lv_2_blocks, project)) for project in projects_list]

level_3_sorting_blocks = [len(level_sorting_blocks_func(lv_3_blocks, project)) for project in projects_list]

level_4_sorting_blocks = [len(level_sorting_blocks_func(lv_4_blocks, project)) for project in projects_list]

Combined_data['Level 1 Blocks'] = level_1_sorting_blocks
Combined_data['Level 2 Blocks'] = level_2_sorting_blocks
Combined_data['Level 3 Blocks'] = level_3_sorting_blocks
Combined_data['Level 4 Blocks'] = level_4_sorting_blocks

Combined_data['Level 1 Block Score'] = Combined_data['Level 1 Blocks'] * 1 * 0.05
Combined_data['Level 2 Block Score'] = Combined_data['Level 2 Blocks'] * 2 * 0.15
Combined_data['Level 3 Block Score'] = Combined_data['Level 3 Blocks'] * 3 * 0.3
Combined_data['Level 4 Block Score'] = Combined_data['Level 4 Blocks'] * 4 * 0.5

Combined_data['Block Complexity Score'] = np.round(
                                              Combined_data['Level 1 Block Score'] +
                                              Combined_data['Level 2 Block Score'] +
                                              Combined_data['Level 3 Block Score'] +
                                              Combined_data['Level 4 Block Score']
                                              )

In [ ]:
# Visualizations for blocks

block_visual = Combined_data.groupby('Blocks Count')[['Project Name']].agg('count').reset_index()

sns.histplot(data=block_visual, y='Blocks Count', x = 'Project Name')

In [ ]:
# Now we can calculate the overall complexity score.
# We average them because the Block Score and Component Scores should have the same weight of 0.5.
Combined_data['Overall Complexity Score'] = (Combined_data['Block Complexity Score'] + Combined_data['Component Complexity Score']) / 2
Combined_data[['Project Name', 'Overall Complexity Score']].sort_values('Overall Complexity Score', ascending=False)

In [ ]:
# Blocks Table
Combined_data[['Project Name', 'Blocks Count', 'Level 1 Blocks', 'Level 2 Blocks', 'Level 3 Blocks', 'Level 4 Blocks', 'Block Complexity Score']].sort_values('Block Complexity Score', ascending=False).head(10)